In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/COMOFOD1/train',target_size=(224, 224),batch_size=32,class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory('/content/drive/My Drive/COMOFOD1/val',target_size=(224, 224),batch_size=32,class_mode='categorical')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/COMOFOD1/test',target_size=(224, 224),batch_size=32,class_mode='categorical')

Found 7004 images belonging to 2 classes.
Found 2009 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen1 = ImageDataGenerator(rescale=1./255)
validation_datagen1 = ImageDataGenerator(rescale=1./255)
train_generator1 = train_datagen1.flow_from_directory('/content/drive/My Drive/COMOFOD1/train',target_size=(299, 299),batch_size=32,class_mode='categorical')
validation_generator1 = validation_datagen1.flow_from_directory('/content/drive/My Drive/COMOFOD1/val',target_size=(299, 299),batch_size=32,class_mode='categorical')
test_datagen1 = ImageDataGenerator(rescale=1./255)
test_generator1 = test_datagen1.flow_from_directory('/content/drive/My Drive/COMOFOD1/test',target_size=(299, 299),batch_size=32,class_mode='categorical')

Found 7004 images belonging to 2 classes.
Found 2009 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
print(validation_generator.image_shape)
print(validation_generator1.image_shape)

(224, 224, 3)
(299, 299, 3)


In [2]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Flatten,UpSampling2D,Reshape
from tensorflow.keras.applications import vgg16,vgg19,ResNet50,InceptionV3,InceptionResNetV2
from keras.applications.densenet import DenseNet201,DenseNet121
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.applications.mobilenet import MobileNet
import scipy
import os
import cv2
from PIL import Image
import numpy as np
#from keras.applications.resnet50 import ResNet50, preprocess_input

In [18]:
vgg=vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
vgg.summary()
for layer in vgg.layers[:-5]:
  print(layer.name)
  layer.trainable=False
x=vgg.layers[-1].output
print(x.shape)

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
#vgg16
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_vgg16.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(x)  
output1=Dense(2,activation="softmax")(output1) 

modelvgg=Model([vgg.input],output1)
modelvgg.compile(loss=['categorical_crossentropy'],optimizer=Adam(learning_rate=1e-5),metrics=["acc"])
print(modelvgg.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
#vgg16
history = modelvgg.fit(
train_generator,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator,validation_steps=63,callbacks=[es, mc])

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [3]:
################################
import tensorflow as tf
dense=DenseNet201(weights='imagenet',include_top=False,input_shape=(224,224,3))
dense.summary()
#dense._name = "dense"
for layer in dense.layers[:-5]:
  print(layer.name)
  layer.trainable=False
y=dense.layers[-1].output
print(y.shape)

74836368/74836368 [==============================] - 1s 0us/step
Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                       

In [4]:
#denseNet201
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_densenet201.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(y)  
output1=Dense(2,activation="softmax")(output1) 

modeldense=Model([dense.input],output1)
modeldense.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modeldense.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [4]:
#densenet201
history = modeldense.fit(
train_generator,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator,validation_steps=63,callbacks=[es, mc])

In [4]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [4]:
print(loss)
print("***************************")
print(val_loss)

In [4]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [4]:
print(acc)
print("*********************************")
print(val_acc)

In [5]:
################################
import tensorflow as tf
resnet=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
resnet.summary()
for layer in resnet.layers[:-5]:
  #model._name = "new_name"
  layer._name = "res"+layer.name
  print(layer.name)
  #layer._name = "res"+layer.name
  layer.trainable=False
z=resnet.layers[-1].output
print(z.shape)

94765736/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [6]:
#resnet50
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_resnet.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(z)  
output1=Dense(2,activation="softmax")(output1) 

modelresnet=Model([resnet.input],output1)
modelresnet.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modelresnet.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 resinput_2 (InputLayer)        [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 resconv1_pad (ZeroPadding2D)   (None, 230, 230, 3)  0           ['resinput_2[0][0]']             
                                                                                                  
 resconv1_conv (Conv2D)         (None, 112, 112, 64  9472        ['resconv1_pad[0][0]']           
                                )                                                                 
                                                                                            

In [6]:
#resnet50 
history = modelresnet.fit(
train_generator,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator,validation_steps=63,callbacks=[es, mc])

In [6]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [6]:
print(loss)
print("***************************")
print(val_loss)

In [6]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [6]:
print(acc)
print("*********************************")
print(val_acc)

In [7]:
import tensorflow as tf
import numpy as np
resnetinc=InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(299,299,3))
resnetinc.summary()
for layer in resnetinc.layers[:-5]:
  #print(layer.name)
  layer.trainable=False
u=resnetinc.layers[-1].output
#x=Reshape((16,4,2048))(x)
u= tf.image.resize(u, [7,7])

print(u.shape)

219055592/219055592 [==============================] - 2s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)  

In [9]:
#InceptionResNetV2
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_resnetinc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(u)  
output1=Dense(2,activation="softmax")(output1) 

modelresnetinc=Model([resnetinc.input],output1)
modelresnetinc.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modelresnetinc.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [9]:
#InceptionResNetV2
history = modelresnetinc.fit(
train_generator1,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator1,validation_steps=63,callbacks=[es, mc])

In [9]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [9]:
print(loss)
print("***************************")
print(val_loss)

In [9]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [9]:
print(acc)
print("*********************************")
print(val_acc)

In [10]:
import tensorflow as tf
import numpy as np
inc=InceptionV3(weights='imagenet',include_top=False,input_shape=(299,299,3))
inc.summary()
for layer in inc.layers[:-5]:
  #print(layer.name)
  layer.trainable=False
v=inc.layers[-1].output
#x=Reshape((16,4,2048))(x)
v= tf.image.resize(v, [7,7])

print(v.shape)

87910968/87910968 [==============================] - 0s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_203 (Conv2D)            (None, 149, 149, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_203 (Batch  (None, 149, 149, 32  96         ['conv2d_203[0][0]']             
 Normalization)       

In [11]:
#InceptionV3
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_inc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(v)  
output1=Dense(2,activation="softmax")(output1) 

modelinc=Model([inc.input],output1)
modelinc.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modelinc.summary())

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_203 (Conv2D)            (None, 149, 149, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_203 (Batch  (None, 149, 149, 32  96         ['conv2d_203[0][0]']             
 Normalization)                 )                                                           

In [11]:
#InceptionV3
history = modelinc.fit(
train_generator1,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator1,validation_steps=63,callbacks=[es, mc])

In [11]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [11]:
print(loss)
print("***************************")
print(val_loss)

In [11]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [11]:
print(acc)
print("*********************************")
print(val_acc)

In [13]:
import tensorflow as tf
mob = tf.keras.applications.MobileNet(
    weights="imagenet",  # load weights pretrained on the ImageNet
    input_shape=(224, 224, 3),
    include_top=False  # do not include the ImageNet classifier at the top
)
mob.summary()
for layer in mob.layers[:-5]:
  print(layer.name)
  layer.trainable=False
t=mob.layers[-1].output
print(t.shape)

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [16]:
#mobilenet
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_mobilenet.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(t)  
output1=Dense(2,activation="softmax")(output1) 

modelmob=Model([mob.input],output1)
modelmob.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modelmob.summary())

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [ ]:
#mobilenet
history = modelmob.fit(
train_generator,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator,validation_steps=63,callbacks=[es, mc])

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [20]:
###########################################   API with 6 models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_sixmodel.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
output = layers.concatenate([x,y,z,t,u,v], axis=3,name='myconcate')
output=GlobalAveragePooling2D()(output)  
output=Dense(2,activation="softmax")(output) 

model=Model([vgg.input,dense.input,resnet.input,mob.input,resnetinc.input,inc.input],output)
model.compile(loss=['categorical_crossentropy'],optimizer=Adam(learning_rate=1e-5),metrics=["acc"])
#print(model.summary())

In [ ]:
print(model.summary())

In [ ]:
def combine_data_generator(X1,X2):
    while True:
        x1i=X1.next()
        x2i=X2.next()     
        yield ([x1i[0],x1i[0],x1i[0],x1i[0],x2i[0],x2i[0]],x1i[1])
                
com=combine_data_generator(train_generator,train_generator1)
print(com)
com1=combine_data_generator(validation_generator,validation_generator1)

<generator object combine_data_generator at 0x7f8d6cd23eb0>


In [21]:
###########merging 6 model 10-5
history = model.fit(
com,
steps_per_epoch=219,
epochs=50,
validation_data=com1,validation_steps=63,callbacks=[es, mc])

In [ ]:
model.save('/content/drive/My Drive/finalsixmodel.h5')

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [24]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('/content/drive/My Drive/finalsixmodel.h5')

In [26]:
################################################## testing on one image for pixel forgery detection
test1_datagen1 = ImageDataGenerator(rescale=1./255)
test1_generator1 = test1_datagen1.flow_from_directory('/content/drive/My Drive/COMOFOD1/test1',target_size=(224, 224),batch_size=32,class_mode='categorical')
test1_datagen2 = ImageDataGenerator(rescale=1./255)
test1_generator2 = test1_datagen2.flow_from_directory('/content/drive/My Drive/COMOFOD1/test1',target_size=(299, 299),batch_size=32,class_mode='categorical')



Found 1 images belonging to 2 classes.
Found 1 images belonging to 2 classes.


In [ ]:
com22=combine_data_generator(test1_generator1,test1_generator2)

In [ ]:
pred=model.predict(com22,steps =1)
print(pred[0])

1/1 [==============================] - 2s 2s/step
[0.00343231 0.9965677 ]


In [26]:
from matplotlib.patches import Rectangle
from skimage import feature
from skimage.feature.peak import peak_local_max
import cv2 as cv


xx,yy=test1_generator1.next()
img=xx[0]
plt.figure(figsize=(4,4))
plt.axis('off')
plt.imshow(xx[0])
plt.show()

pred=model.predict(com22,steps =1)
print(pred)

last_layer_weights=model.layers[-1].get_weights()[0]
#print(last_layer_weights.shape)
last_layer_weights_for_pred=last_layer_weights[:,0]
#print(last_layer_weights_for_pred)

last_conv_model=Model(model.input,model.get_layer("myconcate").output)
#print("*******************",last_conv_model)


#print("np.newaxis",np.newaxis)
last_conv_output=last_conv_model.predict(com22,steps=1)
  
last_conv_output=np.squeeze(last_conv_output)
#print("last_conv_output",last_conv_output.shape)

h=32
w=32
print("h and w",h,w)
upsampled_last_conv_output=scipy.ndimage.zoom(last_conv_output,(h,w,1),order=1)
print(upsampled_last_conv_output.shape)
plt.figure(figsize=(4,4))
plt.axis('off')
plt.imshow(upsampled_last_conv_output[:,:,9087])
#print("upsampled_last_conv_output",upsampled_last_conv_output.shape)
#print(last_layer_weights_for_pred.shape)
heat_map=np.dot(upsampled_last_conv_output.reshape((224*224,9088)),
                  last_layer_weights_for_pred).reshape(224,224)


peak_coords=peak_local_max(heat_map,num_peaks=1,threshold_rel=0.5,min_distance=10)

plt.imshow(img.astype("float32").reshape(img.shape[0],img.shape[1],3))
plt.imshow(heat_map,cmap='jet',alpha=0.60)

for i in range(0,peak_coords.shape[0]):
    print(i)
    y=peak_coords[i,0]
    x=peak_coords[i,1]
    plt.gca().add_patch(Rectangle((x-20,y-20),40,40,linewidth=0,edgecolor='r',facecolor='g', alpha=0.4))

plt.figure(figsize=(4,4))
plt.axis('off')
plt.imshow(xx.astype("float32").reshape(224,224,3))

plt.imshow(heat_map,cmap='jet',alpha=0.60)

fig=plt.figure(figsize=(4,4))
plt.axis('off')
plt.imshow(heat_map,cmap='jet',alpha=0.60)

fig.savefig(r"/content/drive/My Drive/heatmap.png", dpi=fig.dpi)

In [ ]:
################################################theresholding
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
pppp=r"/content/drive/My Drive/heatmap.png"
im=cv.imread(pppp)
imnew=cv.imread(pppp)
#im=cv.cvtColor(im, cv.COLOR_BGR2RGB)
im1=cv.imread(pppp)
#im1=cv.cvtColor(im1, cv.COLOR_BGR2RGB)
im3=cv.imread(pppp)
#im3=cv.cvtColor(im3, cv.COLOR_BGR2RGB)
#cv.imshow('OriginalImage',im)
b,g,r=cv.split(im)
A=np.uint8(b)
B=np.uint8(g)
C=np.uint8(r)
s=im.shape
im2=np.zeros(s,dtype=np.uint8)+0
#im2=im.copy()
im2[:,:]=0
for i in range(0,s[0]):
    for j in range(0,s[1]):
        
        if C[i,j]>=210 and B[i,j]>=179 and A[i,j]==179:
            im2[i,j]=255
            
        
cv.imwrite(r"/content/drive/My Drive/binarymask.png",im2)

import matplotlib.pyplot as plt
im=cv.imread(r"/content/drive/My Drive/binarymask.png")
fig=plt.figure(figsize=(4,4))
plt.axis('off')
plt.imshow(im)



g=cv.imread(r"/content/drive/My Drive/groundtruth.png",0)
print(im.shape)
print(g.shape)
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(0, im.shape[0]):
    for j in range(0, im.shape[1]):
        
        if im[i][j]==0 and g[i][j]==0:
            TP=TP+1;
        elif im[i][j]==255 and g[i][j]==255:
            TN=TN+1;
        elif im[i][j]==255 and g[i][j]==0:
            FN=FN+1;
        elif im[i][j]==0 and g[i][j]==255:
            FP=FP+1;

precision = TP/(TP+FP)
recall = TP/(TP+FN)
accuracy = (TP+TN)/(TP+FP+TN+FN)

fpr=FP/(FP+TN)
iou=TP/(TP+FN+FP)
f1=2*precision*recall/(precision+recall)


print('Accuracy,p,r,f1,fpr,iou', accuracy,precision,recall,f1,fpr,iou)

print("accuracy calculated!")